In [1]:
import pandas as pd
import numpy as np
import os
import re
import string
import tqdm
import spacy

from tqdm.notebook import tqdm_notebook

tqdm_notebook.pandas()
nlp = spacy.load('en_core_web_sm')

C:\ProgramData\Miniconda3\envs\AIOps\lib\site-packages\tqdm\std.py:706: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
short_desc = {
    'Short description', 'short_description', 'BRIEF_DESCRIPTION',
}

assgn_grp = {
    'Assignment group', 'assignment_group', 'ASSIGNMENT_GROUP'
}

In [3]:
path = '../data/'

In [4]:
files = [
    path + file for file in os.listdir(path) if file.endswith('.csv')
]
files

['../data/Cococola_SRsCategorizedDump_1.csv',
 '../data/final_data.csv',
 '../data/Nokia_Incident_v2.csv',
 '../data/Nokia_Service Request.csv',
 '../data/Olympics_Incident.csv',
 '../data/OTSUKA_Inc_6 months_1 .csv',
 '../data/Pfizer Ticket Data _Aug19_Jan20 YT Graph.csv',
 '../data/SAP AMS Incidents 20200630 v2.csv']

In [5]:
def create_data(files: list, assgn: set, desc: set) -> pd.DataFrame:

    _descriptions, _assgn_groups = list(), list()

    for file in files:
        print(f'Processing -> {file}')
        df = pd.read_csv(file)
        available_cols = set(df.columns)

        descriptions = available_cols.intersection(desc)
        assgn_grps = available_cols.intersection(assgn)

        if descriptions and assgn_grps:
            
            _desc = df[descriptions].values.tolist()
            _asgn_grp = df[assgn_grps].values.tolist()
            
            _descriptions.extend(_desc)
            _assgn_groups.extend(_asgn_grp)
            
        else:
            print('One of the required cols is missing !')
            print(f'Found only {descriptions} {assgn_grp}')

    print(len(_descriptions), len(_assgn_groups))
    
    try:
        new_df = pd.DataFrame({
            'short_descriptions': _descriptions,
            'assignment_groups': _assgn_groups
        })
        
        new_df['short_descriptions'] = new_df['short_descriptions'].apply(lambda x: x[0])
        new_df['assignment_groups'] = new_df['assignment_groups'].apply(lambda x: x[0])
        
        return new_df
    except ValueError as e:
        print('Error: ', e)

In [6]:
result = create_data(files=files, assgn=assgn_grp, desc=short_desc)

Processing -> ../data/Cococola_SRsCategorizedDump_1.csv
Processing -> ../data/final_data.csv
One of the required cols is missing !
Found only set() {'assignment_group', 'Assignment group', 'ASSIGNMENT_GROUP'}
Processing -> ../data/Nokia_Incident_v2.csv
Processing -> ../data/Nokia_Service Request.csv


C:\ProgramData\Miniconda3\envs\AIOps\lib\site-packages\IPython\core\interactiveshell.py:3337: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


One of the required cols is missing !
Found only {'short_description'} {'assignment_group', 'Assignment group', 'ASSIGNMENT_GROUP'}
Processing -> ../data/Olympics_Incident.csv
One of the required cols is missing !
Found only {'short_description'} {'assignment_group', 'Assignment group', 'ASSIGNMENT_GROUP'}
Processing -> ../data/OTSUKA_Inc_6 months_1 .csv


C:\ProgramData\Miniconda3\envs\AIOps\lib\site-packages\IPython\core\interactiveshell.py:3337: DtypeWarning: Columns (15,20,21,25,29,31,40,48) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Processing -> ../data/Pfizer Ticket Data _Aug19_Jan20 YT Graph.csv
Processing -> ../data/SAP AMS Incidents 20200630 v2.csv
166049 166049


In [7]:
result.tail()

,short_descriptions,assignment_groups
166044,Data files per our call,PZC-FJ-ServiceDesk
166045,Approver Matrix,PZC-FJ-ServiceDesk
166046,a new Product Hierarchy Level 5 and 6: Vegeta...,PZC-FJ-ServiceDesk
166047,[SAP] Create SAP account in PROD for Charala...,PZC-FJ-ServiceDesk
166048,APO Daily and Monthly Chains to be triggere...,PZC-FJ-ServiceDesk


In [8]:
result.to_csv('../data/final_data.csv', index=False)